In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os


# Part 1: Method-ML Algorithams

More Deatils :
https://www.dataquest.io/blog/top-10-machine-learning-algorithms-for-beginners/

https://www.analyticsvidhya.com/blog/2017/09/common-machine-learning-algorithms/

In [ ]:
#ML Librarires 
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

import seaborn as sns
import missingno as msno
import plotly.express as px
import  matplotlib.pyplot as plt


plt.style.use('seaborn')
%matplotlib inline

# Data Read

In [ ]:
df = pd.read_csv('../input/star-type-classification/Stars.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df=df.sample(frac=1).reset_index(drop=True)

Background: The idiomatic way to do this with Pandas is to use the .sample method of your dataframe to sample all rows without replacement https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows

If you wish to shuffle your dataframe in-place and reset the index, you could do e.g.

df = df.sample(frac=1).reset_index(drop=True)

Here, specifying drop=True prevents .reset_index from creating a column containing the old index entries.

# Exploratory Data Analysis & Data Pre-Processing

https://towardsdatascience.com/exploratory-data-analysis-8fc1cb20fd15

In [ ]:
pd.DataFrame(df.isnull().sum(), columns=["Null Count"])

In [ ]:
#Or 
#missing values cheacking
df.apply(lambda x: sum(x.isnull()),axis=0)

In [ ]:
from sklearn import preprocessing

In [ ]:
#Label Encoding

le = preprocessing.LabelEncoder()

# columns to select for encoding
selected_col = ['Color','Spectral_Class']

le.fit(df[selected_col].values.flatten())

df[selected_col] = df[selected_col].apply(le.fit_transform)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
#Plotting data 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.express as px

plt.figure(figsize=(20,14))
sns.heatmap(df.corr(),annot=True,linecolor='green',linewidths=3,cmap = 'plasma')
#Data Cor-Relation

In [ ]:
#Box Plotting All features distribution corresponding Target column
i=1
plt.figure(figsize=(40,40))
for c in df.columns[:]:
    plt.subplot(3,3,i)
    plt.title(f"Boxplot of {c}",fontsize=16)
    plt.yticks(fontsize=12)
    plt.xticks(fontsize=12)
    sns.boxplot(y=df[c],x=df['Type'])
    i+=1
plt.show()

#Skewness of Data

In [ ]:
# the data columns
cols = ['Temperature', 'L', 'R', 'A_M', 'Color', 'Spectral_Class']
target = ["Type"]

In [ ]:
import scipy.stats as stats
pd.set_option('display.max_columns', None)
plt.style.use('ggplot')

In [ ]:
fig,ax = plt.subplots(3,2, figsize=(16, 12))
ax = ax.flatten()
i = 0
for col in cols:
    skew = df[col].skew()
    sns.distplot(df[col], ax = ax[i], fit= stats.norm, kde=False, label='Skew = %.3f' %(skew))
    ax[i].legend(loc='best')
    i += 1
plt.show()

None of the features are normally distributed and some have outliers

Note: Outlier treatment maybe done to check impact on classification

Deatils: https://www.analyticsvidhya.com/blog/2020/07/what-is-skewness-statistics/

In [ ]:
#checking the target variable countplot

print("class :", df["Type"].unique())
print()

print("Value Count :\n",df["Type"].value_counts())

In [ ]:
len(df["Type"].value_counts())

In [ ]:
#checking the target variable countplot

plt.figure(figsize=(25,15))
sns.set_style('white')
sns.countplot(x='Type', data = df, palette='GnBu')
sns.despine(left=True)

In [ ]:
sns.countplot(df["Type"])
plt.show()

Univariate Box Plot

In [ ]:
df.iloc[:,:-1].boxplot(figsize=(12,6))
plt.show()

Observations:¶
- L is the main component of Df making more than 70% of composition
- Combined temp and L make up around 90%
- A_M, Color, R, spectral  are the least important components

Above box plot confirms the outliers

I prefer to use models without outlier treatment, in many cases it can improve the model performance.
But it also leads to change of information which might alter real/practical situations

Bivariate Box plots

In [ ]:
fig,ax = plt.subplots(3,2, figsize=(16, 12))
ax = ax.flatten()
i = 0
for col in cols:
    sns.boxplot("Type", col, ax = ax[i], data=df)
    ax[i].legend([col], loc='best')
    i += 1
plt.tight_layout()
plt.show()

In [ ]:
#Scatter Matrix
pd.plotting.scatter_matrix(df.iloc[:,:-1], c=df.iloc[:,-1], figsize=(20, 20), marker='o')
plt.legend(df["Type"].unique())
plt.show()

In [ ]:
#Pairplot
sns.pairplot(df, hue='Type', diag_kind='hist')
plt.show()

In [ ]:
#Correlation Plot
plt.figure(figsize=(8,6))
corr = df.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corr, annot=True, fmt= '.2f', cmap='YlGnBu', mask=mask)
plt.show()

In [ ]:
#A_M,Color and Spectral have no correlation with Type, 
#which means for some type it maybe high for some low causing cancelling effect

In [ ]:
df.groupby("Type")["Color"].mean()

In [ ]:
df.groupby("Type")["R"].mean()

Statistical Importance Check for Variable

In [ ]:
import statsmodels.api as sm
import statsmodels.stats as sms

for col in cols:
    data = sm.formula.ols(col+"~ Type", data=df).fit()
    pval = sms.anova.anova_lm(data)["PR(>F)"][0]
    print(f"Pval for {col}: {pval}")

Data Preprocessing & Evaluation Functions

In [ ]:
seed = 42

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier,\
                            BaggingClassifier,VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline, Pipeline

In [ ]:
# split the data into train and test
def split_data(X, Y, seed=42, train_size=0.8):
    xtrain, xtest, ytrain, ytest = train_test_split(X, Y, train_size=train_size, random_state = seed, stratify=Y)
    xtrain, xtest = preprocess(xtrain, xtest)
    return (xtrain, xtest, ytrain, ytest)

# preprocess the data for training
def preprocess(x1, x2=None):
    sc = StandardScaler()
    x1 = pd.DataFrame(sc.fit_transform(x1), columns=x1.columns)
    if x2 is not None:
        x2 = pd.DataFrame(sc.transform(x2), columns=x2.columns)
        return (x1,x2)
    return x1

# for model evaluation and training
def eval_model(model, X, Y, seed=1):
    xtrain, xtest, ytrain, ytest = split_data(X, Y)
    model.fit(xtrain, ytrain)
    
    trainpred = model.predict(xtrain)
    trainpred_prob = model.predict_proba(xtrain)
    testpred = model.predict(xtest)
    testpred_prob = model.predict_proba(xtest)
    
    print("Train ROC AUC : %.4f"%roc_auc_score(ytrain, trainpred_prob, multi_class='ovr'))
    print("\nTrain classification report\n",classification_report(ytrain, trainpred))
    
    ### make a bar chart for displaying the wrong classification of one class coming in which other class
    
    print("\nTest ROC AUC : %.4f"%roc_auc_score(ytest, testpred_prob, multi_class='ovr'))
    print("\nTest classification report\n",classification_report(ytest, testpred))
    
def plot_importance(columns, importance):
    plt.bar(columns, importance)
    plt.show()

In [ ]:
#Feature Extraction, Importance & Splitting

Y= df['Type']

X = df.drop(['Type'],axis = 1)

In [ ]:
X_sc = preprocess(X)

Creating array of models

In [ ]:
model_logr = LogisticRegression(random_state=seed,n_jobs=-1)
model_nb = GaussianNB()
model_dt = DecisionTreeClassifier(random_state=seed)
model_dt_bag = BaggingClassifier(model_dt, random_state=seed, n_jobs=-1)
model_ada = AdaBoostClassifier(random_state=seed)
model_gbc = GradientBoostingClassifier(random_state=seed)
model_rf = RandomForestClassifier(random_state=seed, n_jobs=-1)
model_xgb = XGBClassifier(random_state=seed)
model_lgbm = LGBMClassifier(random_state=seed, n_jobs=-1)
model_knn = KNeighborsClassifier(n_jobs=-1)


In [ ]:
models = []
models.append(('LR',model_logr))
models.append(('NB',model_nb))
models.append(('DT',model_dt))
models.append(('Bag',model_dt_bag))
models.append(('Ada',model_ada))
models.append(('GBC',model_gbc))
models.append(('RF',model_rf))
models.append(('XGB',model_xgb))
models.append(('LGBM',model_lgbm))
models.append(('KNN',model_knn))

Running the algorithms

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

results = []
names = []

for name, model in models:
    scores = cross_val_score(model, X_sc, Y, scoring='f1_weighted', cv=cv, n_jobs=-1)
    accuracy = scores.mean()
    std = scores.std()
    print(f"{name} : Mean ROC {accuracy} STD:({std})")
    results.append(scores)
    names.append(name)

Comparison of Models

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.boxplot(results)
ax.set_xticklabels(names)
plt.show()

Only Significant Variables

In [ ]:
X = df.drop(['A_M', 'Color', 'Spectral_Class'], axis=1)
X_sc = preprocess(X)
Y = df['Type']

In [ ]:
results = []
names = []

for name, model in models:
    scores = cross_val_score(model, X_sc, Y, scoring='f1_weighted', cv=cv, n_jobs=-1)
    accuracy = scores.mean()
    std = scores.std()
    print(f"{name} : Mean ROC {accuracy} STD:({std})")
    results.append(scores)
    names.append(name)

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.boxplot(results)
ax.set_xticklabels(names)
plt.show()